In [83]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [96]:
import torch
import torch.nn as nn
from torch.utils.data import random_split
# from torchvision import models

In [85]:
%run func_DL.py

In [86]:
print(torch.cuda.is_available())

False


---

### model

### dataset

In [87]:
from torch.utils.data import Dataset, DataLoader

#---------------------------------------------
#var
path_dir_X = "../data_X"
path_dir_Y = "../data_Y_Task3"
n_test = 100
n_val = 100
batch_size = 100 #5000

#---------------------------------------------
#instance
dataset = CustomDataset(path_dir_X=path_dir_X, path_dir_Y=path_dir_Y, n_test=n_test, n_val=n_val, batch_size=batch_size)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [88]:
dataset

### var, init

In [89]:
#----------------------------
#var (condition)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#----------------------------
#var (train)
num_epochs = 1000
n_print_train_result = 1
val_flag = True

#----------------------------
#init (model)
in_channels = dataset.return_shape_X()[0]
in_length = dataset.return_shape_X()[1]
out_channels = dataset.return_shape_Y()[0]
out_length = dataset.return_shape_Y()[1]
model = model_task3(in_channels, in_length, out_channels, out_length, batch_size).to(device)
#init model weight
model.apply(init_normal_dist)
#----------------------------
#init (optimizer, scheduler)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10,30,50,100,300,500], gamma=0.95)
#----------------------------
#init (loss_func)
#https://neptune.ai/blog/pytorch-loss-functions
loss_func = nn.MSELoss()
#loss_func = nn.L1Loss()

### load model

In [90]:
#load model
#model.load_state_dict(torch.load('../models/model_task3.pth'))

In [91]:
history = {"train_loss": [], "val_loss": []}

for i, (x, y) in enumerate(dataloader):
    x = dataset.change_data_setting_to_train(x)
    y = dataset.change_data_setting_to_train(y)
    print(i,x.shape,y.shape)

0 torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
1 torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
2 torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
3 torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
4 torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
5 torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
6 torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
7 torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
8 torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
9 torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
10 torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
11 torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
12 torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
13 torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
14 torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
15 torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
16 torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
17 torch.Size([100, 12, 500]) torch.Size([100, 75, 1])
18 torch.Size([100, 

In [92]:
import torch
import torch.nn as nn
import torch.optim as optim

class TransformerModel(nn.Module):
    def __init__(self, input_features, model_dim, num_heads, num_layers, seq_length, output_dim=75):
        super(TransformerModel, self).__init__()
        self.input_embed = nn.Linear(input_features, model_dim)
        self.pos_encoder = nn.Parameter(torch.zeros(1, seq_length, model_dim))
        encoder_layers = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.output_layer = nn.Linear(model_dim, output_dim)

    def forward(self, src):
        # print("Input src shape:", src.shape)
        src = self.input_embed(src)
        # print("After embedding shape:", src.shape)
        src += self.pos_encoder
        # print("After pos_encoder shape:", src.shape)
        output = self.transformer_encoder(src)
        output = output.mean(dim=1)
        return self.output_layer(output)
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerModel(input_features=12, model_dim=512, num_heads=8, num_layers=3, seq_length=500, output_dim=75).to(device)

In [93]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for data, target in dataloader:
        data = data.transpose(1, 2).to(device).float()
        target = target.view(-1, 75).to(device).float()
        # print(f'target shape: {target.shape}')
        optimizer.zero_grad()
        output = model(data)
        # print(f' output shape: {output.shape}')
        loss = criterion(output, target)
        print(f'Loss here {loss}')
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

# Train the model
num_epochs = 5

for epoch in range(num_epochs):
    dataloder = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    loss = train_epoch(model, dataloader, optimizer, criterion, device)
    print(f'Epoch {epoch+1}, Loss: {loss:.4f}')

Loss here 0.28956422209739685
Loss here 1.5389354228973389
Loss here 0.39009106159210205
Loss here 0.11593693494796753
Loss here 0.10428684204816818
Loss here 0.10883431136608124
Loss here 0.0948408916592598
Loss here 0.068019337952137
Loss here 0.04966546222567558
Loss here 0.04819556325674057
Loss here 0.049008022993803024
Loss here 0.04860890284180641
Loss here 0.04715702682733536
Loss here 0.04570722207427025
Loss here 0.041860587894916534
Loss here 0.04470888525247574
Loss here 0.04237836226820946
Loss here 0.04200417920947075
Loss here 0.04181491583585739
Loss here 0.04511535167694092
Loss here 0.04606694355607033
Loss here 0.03679347038269043
Loss here 0.041614074259996414
Loss here 0.0377531535923481
Loss here 0.036211226135492325
Loss here 0.03705573454499245
Loss here 0.03999919816851616
Loss here 0.03978952765464783
Loss here 0.03763972222805023
Loss here 0.03996758908033371
Loss here 0.03807364031672478
Loss here 0.03516411408782005
Loss here 0.03427175059914589
Loss here 0

In [ ]:
def validate_epoch(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data, target in dataloader:
            data = data.transpose(1, 2).to(device).float()
            target = target.view(-1, 75).to(device).float()
            output = model(data)
            loss = criterion(output, target)
            total_loss += loss.item()
    return total_loss / len(dataloader)

# Assume validation set is part of your dataset, if not, remove validation components
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

history = {"train_loss": [], "val_loss": []}

for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
    val_loss = validate_epoch(model, val_loader, criterion, device) if val_flag else None
    history["train_loss"].append(train_loss)
    if val_flag:
        history["val_loss"].append(val_loss)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}' + (f', Val Loss: {val_loss:.4f}' if val_flag else ''))

Loss here 0.009548318572342396
Loss here 0.009752094745635986
Loss here 0.00933888927102089
Loss here 0.010239235125482082
Loss here 0.009764899499714375
Loss here 0.008469054475426674
Loss here 0.00886339321732521
Loss here 0.008508721366524696
Loss here 0.00883712898939848
Loss here 0.00906714890152216
Loss here 0.008666840381920338
Loss here 0.008975629694759846
Loss here 0.009716196916997433
Loss here 0.008483927696943283
Loss here 0.008767160587012768
Loss here 0.008574254810810089
Loss here 0.010041382163763046
Loss here 0.00938682071864605
Loss here 0.010001893155276775
Loss here 0.008877569809556007
Loss here 0.00986435916274786
Loss here 0.008681375533342361
Loss here 0.010130626149475574
Loss here 0.009329861029982567
Loss here 0.008625536225736141
Loss here 0.009306787513196468
Loss here 0.010144539177417755
Loss here 0.009352498687803745
Loss here 0.00940596405416727
Loss here 0.008699106052517891
Loss here 0.009487706236541271
Loss here 0.008956042118370533
Loss here 0.008

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(history['train_loss'], label='Train Loss')
if val_flag:
    plt.plot(history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

### train

### result_train

In [94]:
#----------------------------
#make data (train)
train_loss_tensor = torch.stack(history["train_loss"])
train_loss_np = train_loss_tensor.to('cpu').detach().numpy().copy()
#----------------------------
#make data (val)
val_loss_tensor = torch.stack(history["val_loss"])
val_loss_np = val_loss_tensor.to('cpu').detach().numpy().copy()

#----------------------------
#plot
fig = plt.figure(figsize=(12,7))
ax = fig.add_subplot(1,1,1)
ax.plot(train_loss_np, color="black",label="Train")
ax.plot(val_loss_np, color="maroon",label="Validation")
#plot (setting)
ax.tick_params(labelsize=20)
ax.set_xlabel("Epoch", fontsize=30)
ax.set_ylabel("MSE", fontsize=30)
ax.legend(fontsize=25, frameon=False)
ax.set_ylim(0,0.05)
plt.show()

RuntimeError: stack expects a non-empty TensorList

### result_test

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

#----------------------------
# eval
model.eval()
with torch.no_grad():
    #----------------------------
    #forward
    x, y = dataset.return_test_data()
    #----------------------------
    #float32, grad==True
    x = dataset.change_data_setting_to_train(x)
    y = dataset.change_data_setting_to_train(y)
    #----------------------------
    #change the type
    x = x.to(device)
    y = y.to(device)
    #----------------------------
    #forward
    output = model(x)
    #----------------------------
    #change to numpy
    output = output.to('cpu').detach().numpy().copy().flatten()
    y = y.to('cpu').detach().numpy().copy().flatten()
    loss_MSE = mean_squared_error(output, y)
    loss_R2 = r2_score(output, y)
    #----------------------------
    #print loss
    print("MSE: ", loss_MSE)  
    print("R2: ", loss_R2)

### plot_test (true/pred)

In [ ]:
from sklearn.metrics import r2_score # スコア計算
from sklearn.metrics import mean_absolute_error # スコア計算 (MAE)
from sklearn.metrics import mean_squared_error # スコア計算 (MSE)
#----------------
def plot_true_predict_from_y(y_predict_list:pd.DataFrame, y_true_list:pd.DataFrame,
                            title:str, path_save=False) -> None:
    #----------------
    #calc score
    r = np.corrcoef(y_true_list, y_predict_list)[0][1]
    R2 = r2_score(y_true=y_true_list, y_pred=y_predict_list) # 決定係数(R2) #https://bellcurve.jp/statistics/course/9706.html
    MAE = mean_absolute_error(y_true=y_true_list, y_pred=y_predict_list) # 平均絶対誤差(MAE)
    RMSE = np.sqrt(mean_squared_error(y_true=y_true_list, y_pred=y_predict_list)) # 二乗平均平方根誤差(RMSE)
    
    #----------------
    #fig, ax
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(1,1,1)
    #----------------
    #plot scatter
    #ax.scatter(x=y_predict_list, y=y_true_list, s=40, c="black", marker="o", zorder=10)
    ax.plot(y_predict_list, y_true_list, c="black", marker='.', linestyle="", ms=3, zorder=10)
    #----------------
    #plot 直線
    x=np.linspace( min(min(y_true_list),min(y_predict_list)), max(max(y_true_list),max(y_predict_list)), 10) #listの足し算は結合
    y=x
    ax.plot(x, y, color = "black")
    #----------------
    #plot text
    plt.text(x=0.5, y=0.94, 
             s="$r$={0}, $R^2$={1}, $MAE$={2}, $RMSE$={3}".format("{:.2f}".format(r),
                                                                 "{:.2f}".format(R2),
                                                                "{:.2f}".format(MAE),
                                                                "{:.2f}".format(RMSE)), 
             fontdict=dict(fontsize=25, color="black"), ha='center', transform=ax.transAxes,
             zorder=20)
    #----------------
    #setting
    ax.tick_params(labelsize = 20)#軸の大きさ
    ax.set_xlabel("True",fontsize=30)
    ax.set_ylabel("Predict",fontsize=30)
    plt.title("{0}".format(title), fontsize=30)
    #----------------
    #save
    if path_save != False:
        plt.savefig(path_save, bbox_inches='tight')
    #----------------
    #show
    plt.show()

In [ ]:
#----------------------------
# eval
model.eval()
with torch.no_grad():
    #----------------------------
    #forward
    x, y = dataset.return_test_data()
    #----------------------------
    #float32, grad==True
    x = dataset.change_data_setting_to_train(x)
    y = dataset.change_data_setting_to_train(y)
    #----------------------------
    #change the type
    x = x.to(device)
    y = y.to(device)
    #----------------------------
    #forward
    output = model(x)
    #----------------------------
    #change to numpy
    output = output.to('cpu').detach().numpy().copy().flatten() * 185
    y = y.to('cpu').detach().numpy().copy().flatten() * 185
    #----------------------------
    #plot
    plot_true_predict_from_y(y_predict_list=output, y_true_list=y, title="", path_save=False) 

### plot_test (sample_all)

In [ ]:
def plot_VmDatas_Task3_all(ActTime, title):
    # plot the Activation Time array
    plt.imshow(ActTime, cmap='jet', interpolation='nearest', aspect='auto')
    plt.title('Activation Time')
    cbar = plt.colorbar()
    plt.grid(visible=True, which='major', color='#666666', linestyle='-')
    plt.minorticks_on()
    # not xticks
    #plt.xticks([])
    plt.grid(visible=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
    plt.title("{0}".format(title), fontsize=20)
    plt.xlabel("Test data ID",fontsize=15)
    plt.ylabel("Number of Activation Map",fontsize=15)
    cbar.set_label('Activation Time', fontsize=15, rotation=270, labelpad=15)
    plt.show()

In [ ]:
#var
n_plot = 3

#----------------------------
# eval
model.eval()
with torch.no_grad():
    #----------------------------
    #forward
    x, y = dataset.return_test_data()
    #----------------------------
    #float32, grad==True
    x = dataset.change_data_setting_to_train(x)
    y = dataset.change_data_setting_to_train(y)
    #----------------------------
    #change the type
    x = x.to(device)
    y = y.to(device)
    #----------------------------
    #forward
    output = model(x)
    #----------------------------
    #change to numpy
    output = output.to('cpu').view(75, dataset.return_n_test()).detach().numpy().copy() * 185
    y = y.to('cpu').view(75, dataset.return_n_test()).detach().numpy().copy() * 185
    #----------------------------
    #plot
    plot_VmDatas_Task3_all(y, title="True")
    plot_VmDatas_Task3_all(output, title="Predict")

### plot_test (sample_each)

In [ ]:
def plot_VmDatas_Task3_each(ActTime, title):
    # plot the Activation Time array
    plt.imshow(ActTime, cmap='jet', interpolation='nearest', aspect='auto')
    plt.title('Activation Time')
    cbar = plt.colorbar()
    plt.grid(visible=True, which='major', color='#666666', linestyle='-')
    plt.minorticks_on()
    # not xticks
    plt.xticks([])
    plt.grid(visible=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
    plt.title("{0}".format(title), fontsize=20)
    #plt.xlabel("Data ID",fontsize=15)
    plt.ylabel("Number of Activation Map",fontsize=15)
    cbar.set_label('Activation Time', fontsize=15, rotation=270, labelpad=15)
    plt.show()

In [ ]:
#var
n_plot = 3

#----------------------------
# eval
model.eval()
with torch.no_grad():
    #----------------------------
    #forward
    x, y = dataset.return_test_data()
    #----------------------------
    #float32, grad==True
    x = dataset.change_data_setting_to_train(x)
    y = dataset.change_data_setting_to_train(y)
    #----------------------------
    #change the type
    x = x.to(device)
    y = y.to(device)
    #----------------------------
    #forward
    output = model(x)
    #----------------------------
    #change to numpy
    output = output.to('cpu').detach().numpy().copy() * 185
    y = y.to('cpu').detach().numpy().copy() * 185
    print(y.shape)
    #----------------------------
    #plot
    for cnt, (data_true, data_pred) in enumerate(zip(y, output)):
        if cnt+1 <= n_plot:
            plot_VmDatas_Task3_each(data_true, title="True")
            plot_VmDatas_Task3_each(data_pred, title="Predict")

### save model

In [ ]:
#https://wandb.ai/wandb/common-ml-errors/reports/How-to-Save-and-Load-Models-in-PyTorch--VmlldzozMjg0MTE
#save model
#torch.save(model.state_dict(), '../models/model_task3.pth')

---

# Test (CustomDataset)

In [ ]:
import os, random
from torch.utils.data import Dataset, DataLoader
from typing import Union

#---------------------------------------------
# custom dataset
#https://discuss.pytorch.org/t/custom-data-loader-for-big-data/129361
class CustomDataset(Dataset):
    def __init__(self, path_dir_X:str, path_dir_Y:str, n_test:Union[int,float], n_val:Union[int,float], batch_size:int): # n_test -> float:ratio of test, int:number of test
        #-----------------
        # batch_size
        self.batch_size = batch_size
        # path_dir_X, path_dir_Y
        self.path_dir_X = path_dir_X
        self.path_dir_Y = path_dir_Y
        # list_file_name_all
        self.list_file_name_all = os.listdir(path_dir_X)
        # n_data_all
        self.n_data_all = len(self.list_file_name_all)
        #check
        if len(os.listdir(path_dir_X)) != len(os.listdir(path_dir_Y)):
            raise ValueError("error!!!")
        if len(set(os.listdir(path_dir_X)) - set(os.listdir(path_dir_Y))) != 0:
            raise ValueError("error!!!")
        #-----------------
        # suffle
        random.shuffle(self.list_file_name_all)
        #-----------------
        # n_test
        if type(n_test)==int:
            self.n_test = n_test
        elif type(n_test)==float:
            self.n_test = int(len(self.list_file_name_all)*n_test)
        else:
            raise ValueError("error!!!")
        # n_val
        if type(n_val)==int:
            self.n_val = n_val
        elif type(n_val)==float:
            self.n_val = int(len(self.list_file_name_all)*n_val)
        else:
            raise ValueError("error!!!")
        #check
        if self.n_data_all <= self.n_test+self.n_val:
            raise ValueError("error!!!")
        #-----------------
        # list_file_name_test / _val / _train
        self.list_file_name_test = self.list_file_name_all[:self.n_test]
        self.list_file_name_val = self.list_file_name_all[self.n_test:self.n_test+self.n_val]
        self.list_file_name_train = self.list_file_name_all[self.n_test+self.n_val:]
        
    def __len__(self):
        return len(self.list_file_name_train)
    
    def __getitem__(self, x):
        return self.getdata(list_file_name=self.list_file_name_train, index=x)
    
    def getdata(self, list_file_name, index):
        #file_name
        file_name = list_file_name[index]
        #data_X
        path_file_X = "{0}/{1}".format(self.path_dir_X, file_name)
        data_X = np.load(path_file_X, allow_pickle=True)
        data_X = torch.from_numpy(data_X).to(torch.float32)
        #data_Y
        path_file_Y = "{0}/{1}".format(self.path_dir_Y, file_name)
        data_Y = np.load(path_file_Y, allow_pickle=True)
        data_Y = torch.from_numpy(data_Y).to(torch.float32)
        #return
        return data_X, data_Y
    
    def return_n_data_all(self):
        return self.n_data_all
    
    def return_n_test(self):
        return self.n_test
    
    def return_n_val(self):
        return self.n_val
    
    def return_n_train(self):
        return self.n_data_all - self.n_val - self.n_test
    
    def return_batch_size(self):
        return self.batch_size
    
    def return_shape_X(self):
        data_sample = self.getdata(self.list_file_name_all, 0)[0]
        return data_sample.shape
    
    def return_shape_Y(self):
        data_sample = self.getdata(self.list_file_name_all, 0)[1]
        return data_sample.shape
    
    def return_test_data(self):
        #https://www.tutorialspoint.com/how-to-join-tensors-in-pytorch
        data_X_test = torch.stack([self.getdata(self.list_file_name_test, i)[0] for i in range(self.n_test)])
        data_Y_test = torch.stack([self.getdata(self.list_file_name_test, i)[1] for i in range(self.n_test)])
        return data_X_test, data_Y_test
    
    def return_val_data(self):
        #https://www.tutorialspoint.com/how-to-join-tensors-in-pytorch
        data_X_val = torch.stack([self.getdata(self.list_file_name_val, i)[0] for i in range(self.n_val)])
        data_Y_val = torch.stack([self.getdata(self.list_file_name_val, i)[1] for i in range(self.n_val)])
        return data_X_val, data_Y_val


#---------------------------------------------
#var
path_dir_X = "../data_X"
path_dir_Y = "../data_Y_Task3"
n_test = 100
n_val = 100
batch_size = 10000

#---------------------------------------------
#instance
dataset = CustomDataset(path_dir_X=path_dir_X, path_dir_Y=path_dir_Y, n_test=n_test, n_val=n_val, batch_size=batch_size)
dataloder = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
for i, (data_X, data_Y) in enumerate(dataloder):
   # print(data_X.shape, data_Y.shape)
   print(data_Y)

In [ ]:
dataset.return_n_data_all()

In [ ]:
dataset.return_n_test()

In [ ]:
dataset.return_n_val()

In [ ]:
dataset.return_val_data()[1].shape

In [ ]:
dataset.return_test_data()[1].shape

In [ ]:
x,y = dataset.return_val_data()

---